In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import nltk
import warnings
%matplotlib inline

warnings.filterwarnings('ignore')

In [2]:
from google.colab import files
uploaded = files.upload()

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
import io
 
data = pd.read_csv(io.BytesIO(uploaded['df_limpio_v2.csv']))
data.head()

In [3]:
#get data
path = input('path to the csv: ')
data = pd.read_csv(f"{path}")
data.head()

#/Users/danielriojas/Documents/Random_2022/df_limpio_v1.csv
#/Users/danielriojas/Downloads/twitter_reddit_clean.csv 

path to the csv: /Users/danielriojas/Downloads/twitter_reddit_clean.csv


,Unnamed: 0,text,label
0,0,god morning stil excited schol people,0
1,1,never felt low life wish easy way kil like fal...,1
2,2,haha love hate relationship yelow great doesnt...,0
3,3,day legendary season finale quot met mother quot,0
4,4,completely oposite girl say swet stuf thought ...,1


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25368 entries, 0 to 25367
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  25368 non-null  int64 
 1   text        25284 non-null  object
 2   label       25368 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 594.7+ KB


In [5]:
data = data.dropna()

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25284 entries, 0 to 25367
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  25284 non-null  int64 
 1   text        25284 non-null  object
 2   label       25284 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 790.1+ KB


In [7]:
positive_data = data[data.label == 0]

In [8]:
depression_data = data[data.label==1]

In [9]:
sample_positive_data = positive_data.sample( n  = 2345)

In [10]:
sample_data = sample_positive_data.append(depression_data)

In [11]:
sample_data =  sample_data.sample(frac=1)

In [12]:
text_col = input('data: ')
X = sample_data[f'{text_col}']

target_col = input('target: ')
y = sample_data[f'{target_col}']

data: text
target: label


In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

In [14]:
nltk.download('punkt')

X_train = [nltk.tokenize.word_tokenize(str(_)) for _ in X_train]
X_test = [nltk.tokenize.word_tokenize(str(_)) for _ in X_test]

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/danielriojas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [15]:
import gensim.downloader as api

info = api.info()

In [16]:
model = api.load('glove-twitter-25')

[==================================================] 100.0% 104.8/104.8MB downloaded


In [17]:
model.most_similar('happy')

[('birthday', 0.9577818512916565),
 ('thank', 0.937666654586792),
 ('welcome', 0.93361496925354),
 ('love', 0.9176183342933655),
 ('miss', 0.9164500832557678),
 ('hello', 0.9158351421356201),
 ('thanks', 0.915008544921875),
 ('merry', 0.9053249359130859),
 ('bless', 0.902732253074646),
 ('wish', 0.9013164043426514)]

In [18]:
model.most_similar('sad')

[('swear', 0.9130125641822815),
 ('feel', 0.912410318851471),
 ('same', 0.9080260992050171),
 ('wrong', 0.9030309319496155),
 ('meant', 0.9027895927429199),
 ('true', 0.9025242924690247),
 ('reason', 0.9024712443351746),
 ('remember', 0.9011679291725159),
 ('i', 0.8995794057846069),
 ('know', 0.899130642414093)]

In [19]:
model.most_similar('help')

[('save', 0.9302789568901062),
 ('must', 0.9283666610717773),
 ('need', 0.9236305356025696),
 ('anyone', 0.9137064218521118),
 ('give', 0.9123104214668274),
 ('us', 0.9109662771224976),
 ('support', 0.910312831401825),
 ('care', 0.9066923260688782),
 ('spread', 0.9033982157707214),
 ('make', 0.902855396270752)]

In [20]:
model.most_similar('depression')

[('anxiety', 0.9597738981246948),
 ('illness', 0.9199443459510803),
 ('causes', 0.9046310782432556),
 ('suffering', 0.8942320346832275),
 ('disease', 0.890959620475769),
 ('sickness', 0.8868004679679871),
 ('symptoms', 0.8831015825271606),
 ('boredom', 0.8755505084991455),
 ('disorder', 0.8740999698638916),
 ('cure', 0.8725259900093079)]

In [21]:
model.most_similar('suicide')

[('torture', 0.9261065721511841),
 ('prison', 0.9218338131904602),
 ('abuse', 0.9135087728500366),
 ('rape', 0.9132890701293945),
 ('murder', 0.9060637950897217),
 ('violence', 0.9008667469024658),
 ('attacks', 0.8894689083099365),
 ('death', 0.8759053349494934),
 ('threats', 0.871550977230072),
 ('victim', 0.8707210421562195)]

In [22]:
from tensorflow.keras.preprocessing.text import Tokenizer

tk = Tokenizer()
tk.fit_on_texts(X)

In [23]:
vocab_size = len(tk.word_index)
vocab_size

25173

In [24]:
X.shape , y.shape

((15024,), (15024,))

In [25]:
print(tk.word_index)

{'fel': 1, 'like': 2, 'want': 3, 'life': 4, 'know': 5, 'get': 6, 'time': 7, 'even': 8, 'depresion': 9, 'people': 10, 'thing': 11, 'day': 12, 'would': 13, 'year': 14, 'realy': 15, 'friend': 16, 'one': 17, 'going': 18, 'make': 19, 'anymore': 20, 'think': 21, 'go': 22, 'never': 23, 'much': 24, 'help': 25, 'take': 26, 'feling': 27, 'god': 28, 'could': 29, 'anything': 30, 'work': 31, 'thought': 32, 'back': 33, 'always': 34, 'way': 35, 'beter': 36, 'stil': 37, 'everything': 38, 'something': 39, 'got': 40, 'nothing': 41, 'hate': 42, 'ned': 43, 'every': 44, 'se': 45, 'someone': 46, 'job': 47, 'love': 48, 'depresed': 49, 'family': 50, 'hapy': 51, 'anyone': 52, 'bad': 53, 'say': 54, 'since': 55, 'talk': 56, 'month': 57, 'care': 58, 'right': 59, 'also': 60, 'ever': 61, 'fucking': 62, 'live': 63, 'kep': 64, 'try': 65, 'alone': 66, 'person': 67, 'anxiety': 68, 'geting': 69, 'end': 70, 'last': 71, 'everyone': 72, 'hard': 73, 'point': 74, 'trying': 75, 'lot': 76, 'schol': 77, 'tired': 78, 'long': 79,

In [26]:
print('Found %s unique tokens.' % len(tk.word_index))

Found 25173 unique tokens.


In [34]:
sequences = tk.texts_to_sequences(X_train)

In [35]:
sequences[:5]

[[18,
  54,
  982,
  717,
  788,
  2,
  58,
  593,
  8505,
  1351,
  2451,
  668,
  126,
  163,
  578,
  39,
  668,
  87,
  10,
  120,
  193,
  32,
  315,
  242,
  13,
  19,
  76,
  160,
  10,
  13,
  2,
  76,
  62,
  10,
  931,
  2763,
  10,
  414,
  73,
  29,
  419,
  104,
  2451,
  1696,
  609,
  81,
  503,
  62,
  2763,
  73,
  159,
  1941,
  686,
  364,
  1300,
  8,
  4,
  640,
  35,
  7598,
  132,
  2210,
  1315,
  8,
  127,
  67,
  931,
  8,
  127,
  106,
  3,
  39,
  4,
  3,
  173,
  148,
  3,
  19,
  10,
  45,
  11,
  19,
  56,
  24,
  48,
  3,
  1759,
  471,
  4,
  288,
  10,
  2812,
  105,
  17,
  214,
  17,
  58,
  17,
  122,
  11,
  87,
  19,
  368,
  3190,
  1540,
  23,
  184,
  852,
  67,
  163,
  23,
  1163,
  596,
  4,
  225,
  5085,
  5677,
  2283,
  10,
  128,
  415,
  104,
  150,
  1672,
  105,
  2190,
  41,
  141,
  129,
  200,
  141,
  72,
  115,
  29,
  31,
  73,
  1894,
  141,
  28,
  18,
  105,
  12,
  150,
  1672,
  104],
 [78,
  27,
  2,
  165,
  844,
  1,
  

In [36]:
#Identify max length
max_length = 0
for review_number in range(len(sequences)):
  numberofwords = len(sequences[review_number])
  if (numberofwords) > (max_length):
    max_length = numberofwords
print(max_length)

3120


In [37]:
#Padding
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [38]:
padded_data = pad_sequences(sequences, maxlen=max_length, padding='post')
y = np.asarray(y_train)

In [40]:
print(f'shape of data tensor: {padded_data.shape}')
print(f'shape of target tensor: {y.shape}')

shape of data tensor: (10516, 3120)
shape of target tensor: (10516,)
